0. Base Preprocessing (Inherited)

In [1]:
def read_csv_safe(path):
    try:
        return pd.read_csv(path)
    except UnicodeDecodeError:
        try:
            return pd.read_csv(path, encoding="cp1252")
        except UnicodeDecodeError:
            return pd.read_csv(path, encoding="latin1")

In [2]:
import pandas as pd
import numpy as np

# ---------
# 1. Paths
# ---------

DATA_DIR = ""

PATH_DIM_BLOCKS = DATA_DIR + "dim_blocks_v1.csv"
PATH_FACT_STORES = DATA_DIR + "fact_stores_v1.csv"
PATH_FACT_MALLS = DATA_DIR + "fact_malls_v1.csv"
PATH_FINANCIALS = DATA_DIR + "store_financials_v1.csv"
PATH_SRI = DATA_DIR + "fact_sri_scores_v1.csv"
PATH_CROSS_VISITS = DATA_DIR + "cross_visits_v1.csv"

# --------------------
# 2. Load raw datasets
# --------------------
dim_blocks = read_csv_safe(PATH_DIM_BLOCKS)
fact_stores = read_csv_safe(PATH_FACT_STORES)
fact_malls = read_csv_safe(PATH_FACT_MALLS)
financials = read_csv_safe(PATH_FINANCIALS)
sri = read_csv_safe(PATH_SRI)
cross_visits = read_csv_safe(PATH_CROSS_VISITS)

# -----------------------
# 3. Basic column cleanup
# -----------------------
def clean_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("ï»¿", "")
    )
    return df

for df in [dim_blocks, fact_stores, fact_malls, financials, sri, cross_visits]:
    clean_columns(df)

# --------------------------
# 4. dim_blocks preprocessing
# --------------------------
dim_blocks = dim_blocks.drop_duplicates(subset=["block_id"])

# GLA patch
if "gla" in dim_blocks.columns:
    dim_blocks["gla"] = (
        dim_blocks.groupby("store_code")["gla"]
        .transform(lambda x: x.fillna(x.median()))
    )

# --------------------------------
# 5. fact_stores preprocessing
# --------------------------------

fact_stores["date"] = pd.to_datetime(
    fact_stores["date"],
    dayfirst=True,
    errors="coerce"
)

fact_malls["date"] = pd.to_datetime(
    fact_malls["date"],
    dayfirst=True,
    errors="coerce"
)


# to avoid division by zero / outdoor stores
fact_stores["people_window_flow"] = fact_stores["people_window_flow"].replace(0, np.nan)

fact_stores["entry_rate"] = (
    fact_stores["people_in"] / fact_stores["people_window_flow"]
)

fact_stores["outdoor_flag"] = (
    fact_stores["people_in"] > fact_stores["people_window_flow"]
)

# --------------------------------
# 6. Financials preprocessing
# --------------------------------
financials = financials.rename(columns={"codstr": "store_code"})

num_cols = ["sales_r12m", "total_costs_r12m"]

for col in num_cols:
    financials[col] = (
        financials[col]
        .astype(str)
        .str.replace(" ", "", regex=False)
        .str.replace(",", ".", regex=False)
    )
    financials[col] = pd.to_numeric(financials[col], errors="coerce")

financials["sales_per_cost"] = (
    financials["sales_r12m"] / financials["total_costs_r12m"].abs()
)

financials["has_financials"] = financials["sales_r12m"].notna().astype(int)


# --------------------------
# 7. SRI preprocessing
# --------------------------
sri["has_sri"] = sri["sri_score"].notna().astype(int)

# ---------------------------------------
# 8. Build master store-level table
# ---------------------------------------
store_master = (
    dim_blocks
    .merge(financials, on="store_code", how="left")
    .merge(sri, on="store_code", how="left")
)

# ---------------------------------------
# 9. Build base store-day table (core)
# ---------------------------------------
store_day = (
    fact_stores
    .merge(dim_blocks, on=["store_code", "mall_id"], how="left")
    .merge(fact_malls, on=["mall_id", "date"], how="left", suffixes=("", "_mall"))
)

# -------------------
# 10. Sanity checks
# -------------------
print("store_master shape:", store_master.shape)
print("store_day shape:", store_day.shape)

store_day.head()


C:\Users\soumy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\soumy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\soumy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\soumy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarnin

store_master shape: (3854, 18)
store_day shape: (1082344, 26)


,date,mall_id,block_id_x,store_code,retailer_id,people_in,people_window_flow,store_average_dwell_time,store_median_dwell_time,shopping_average_dwell_time,...,retailer_code,bl1_label,bl2_label,bl3_label,gla,gla_category,people_in_mall,average_dwell_time,dwell_time_sample,median_dwell_time
0,2024-07-01,19,7064,1300441,50969.0,943,10797.0,37.864189,34.045196,126.012863,...,50969.0,Food & Beverage Services,Service to table restaurants,Italian - Service to table restaurants,291.0,SMALL UNITS,51186.0,64.273182,27516.0,43.3333
1,2024-07-01,7,4844,1108435,88663.0,95,3816.0,33.365112,29.597769,206.777853,...,88663.0,Food & Beverage Services,Fast Meal,Latino-American - Fast Meal,172.0,SMALL UNITS,42536.0,75.168960,18018.0,58.2833
2,2024-07-01,16,1671,600007,504.0,741,41704.0,2.843237,2.635919,61.569458,...,504.0,Bags & Footwear & Accessories,Accessories,"Mixed: small accessories, bags and fancy jewel...",63.0,SMALL UNITS,101274.0,46.767597,33555.0,35.7333
3,2024-07-01,10,4564,1089312,711.0,97,2180.0,4.249951,3.927536,103.858489,...,711.0,Fashion apparel,Family Fashion,Family Fashion,199.6,SMALL UNITS,36889.0,73.247915,15183.0,51.7667
4,2024-07-01,16,1703,1306503,131720.0,342,2591.0,63.461003,58.833670,168.490587,...,131720.0,Food & Beverage Services,Service to table restaurants,Pub & Sports bar,609.0,MSU,101274.0,46.767597,33555.0,35.7333


1. Temporal Rolling Features (Store-level)

In [3]:
# =====================================
# 1. Temporal Rolling Features
# =====================================

# ranking adjustment
store_day = store_day.sort_values(["store_code", "date"])

# -----------------------------
# 1.1 Basic numbers columns
# -----------------------------
base_metrics = [
    "people_in",
    "people_window_flow",
    "entry_rate",
    "store_average_dwell_time",
    "store_median_dwell_time",
    "shopping_average_dwell_time"
]

# -----------------------------
# 1.2 Rolling windows
# -----------------------------
windows = [7, 14, 28]

# -----------------------------
# 1.3 Rolling mean / std
# -----------------------------
for w in windows:
    for col in base_metrics:
        store_day[f"{col}_roll{w}_mean"] = (
            store_day
            .groupby("store_code")[col]
            .transform(lambda x: x.shift(1).rolling(w, min_periods=3).mean())
        )

        store_day[f"{col}_roll{w}_std"] = (
            store_day
            .groupby("store_code")[col]
            .transform(lambda x: x.shift(1).rolling(w, min_periods=3).std())
        )

# -----------------------------
# 1.4 Short-term trend features
# -----------------------------
# Δ7d / Δ14d
for col in base_metrics:
    store_day[f"{col}_chg_7d"] = (
        store_day
        .groupby("store_code")[col]
        .transform(lambda x: x - x.shift(7))
    )

    store_day[f"{col}_pctchg_7d"] = (
        store_day
        .groupby("store_code")[col]
        .transform(lambda x: (x - x.shift(7)) / (x.shift(7).abs() + 1e-6))
    )

# -----------------------------
# 1.5 Volatility / stability
# -----------------------------
for col in base_metrics:
    store_day[f"{col}_cv_14d"] = (
        store_day[f"{col}_roll14_std"] /
        (store_day[f"{col}_roll14_mean"].abs() + 1e-6)
    )

# -----------------------------
# 1.6 Data availability flags
# -----------------------------
for w in windows:
    store_day[f"has_history_{w}d"] = (
        store_day
        .groupby("store_code")["date"]
        .transform(lambda x: x.shift(1).rolling(w).count() >= w)
        .astype(int)
    )

# -----------------------------
# 1.7 Quick sanity check
# -----------------------------
rolling_cols = [c for c in store_day.columns if "roll" in c or "chg" in c]

print("Temporal features created:", len(rolling_cols))
store_day[rolling_cols].describe().T.head(10)

Temporal features created: 48


,count,mean,std,min,25%,50%,75%,max
people_in_roll7_mean,1071707.0,931.152080,1726.047235,16.00000,158.142857,383.142857,937.428571,42461.571429
people_in_roll7_std,1071707.0,344.246961,629.518783,0.00000,62.793312,147.571936,357.910770,20089.844456
people_window_flow_roll7_mean,1063050.0,16773.991920,16352.053980,19.60000,6047.571429,12219.428571,22127.000000,231754.285714
people_window_flow_roll7_std,1063050.0,6083.407001,5601.907130,0.00000,2059.243377,4493.839492,8383.675017,107620.364773
entry_rate_roll7_mean,1063050.0,0.070474,1.421927,0.00095,0.019166,0.035278,0.068716,526.810782
entry_rate_roll7_std,1063050.0,0.029461,2.523919,0.00000,0.003023,0.005659,0.011001,721.341185
store_average_dwell_time_roll7_mean,980770.0,11.909523,13.258742,0.00000,4.102810,6.233507,13.794661,173.442999
store_average_dwell_time_roll7_std,980770.0,2.419411,3.891001,0.00000,0.490224,0.990312,2.832838,229.322598
store_median_dwell_time_roll7_mean,980770.0,9.385086,12.654682,0.00000,2.746892,4.253317,9.099906,160.796200
store_median_dwell_time_roll7_std,980770.0,2.100311,4.031742,0.00000,0.329071,0.644482,2.019798,207.214892


2. Relative Features (Store vs Mall / Peers)

In [4]:
# =====================================
# 2. Relative Features
# =====================================

# -----------------------------
# 2.1 Store vs Mall (same day)
# -----------------------------

store_day["rel_people_in_mall"] = (
    store_day["people_in"] /
    (store_day["people_in_mall"].abs() + 1e-6)
)

store_day["rel_dwell_avg_mall"] = (
    store_day["store_average_dwell_time"] /
    (store_day["average_dwell_time"].abs() + 1e-6)
)

store_day["rel_dwell_median_mall"] = (
    store_day["store_median_dwell_time"] /
    (store_day["median_dwell_time"].abs() + 1e-6)
)



# -----------------------------
# 2.2 Store vs Mall (z-score)
# -----------------------------
# "relative behavior as in the same day"

for col in ["people_in", "store_average_dwell_time"]:
    mall_mean = (
        store_day
        .groupby(["mall_id", "date"])[col]
        .transform("mean")
    )
    mall_std = (
        store_day
        .groupby(["mall_id", "date"])[col]
        .transform("std")
    )

    store_day[f"{col}_z_mall"] = (
        (store_day[col] - mall_mean) / (mall_std + 1e-6)
    )

# -----------------------------
# 2.3 Store vs Same Category (peer)
# -----------------------------
# use bl2_label as peer group

peer_key = ["mall_id", "bl2_label", "date"]

for col in ["people_in", "entry_rate", "store_average_dwell_time"]:
    peer_mean = (
        store_day
        .groupby(peer_key)[col]
        .transform("mean")
    )

    store_day[f"{col}_rel_peer"] = (
        store_day[col] / (peer_mean.abs() + 1e-6)
    )

# -----------------------------
# 2.4 Store vs Own History (short-term)
# -----------------------------

store_day["people_in_rel_hist_7d"] = (
    store_day["people_in"] /
    (store_day["people_in_roll7_mean"].abs() + 1e-6)
)

store_day["entry_rate_rel_hist_7d"] = (
    store_day["entry_rate"] /
    (store_day["entry_rate_roll7_mean"].abs() + 1e-6)
)

# -----------------------------
# 2.5 Availability flags
# -----------------------------
store_day["has_mall_context"] = (
    store_day["people_in_mall"].notna().astype(int)
)

store_day["has_peer_context"] = (
    store_day["bl2_label"].notna().astype(int)
)

# -----------------------------
# 2.6 Quick sanity check
# -----------------------------
relative_cols = [c for c in store_day.columns if c.startswith("rel_") or "_z_" in c or "_rel_" in c]

print("Relative features created:", len(relative_cols))
store_day[relative_cols].describe().T.head(10)


Relative features created: 10


,count,mean,std,min,25%,50%,75%,max
rel_people_in_mall,1082344.0,1.860214e-02,0.037297,0.000047,0.002786,0.007232,0.018552,1.325658e+00
rel_dwell_avg_mall,979429.0,3.553120e+01,15767.880781,0.000000,0.057469,0.091086,0.203004,8.015549e+06
rel_dwell_median_mall,979496.0,7.131773e+02,150312.493309,0.000000,0.055676,0.091499,0.199133,9.359654e+07
people_in_z_mall,1082343.0,-1.421948e-17,0.996733,-1.392663,-0.459240,-0.311144,0.027606,1.277488e+01
store_average_dwell_time_z_mall,980282.0,2.899340e-18,0.991143,-2.455742,-0.559645,-0.379425,0.117074,1.047332e+01
people_in_rel_peer,1076633.0,1.000000e+00,0.858607,0.002615,0.371146,0.845857,1.339387,1.145864e+01
entry_rate_rel_peer,1067773.0,9.999707e-01,0.751411,0.000017,0.505341,0.881927,1.247508,1.629123e+01
store_average_dwell_time_rel_peer,975124.0,9.892617e-01,0.563956,0.000000,0.649997,0.934711,1.179392,1.403324e+01
people_in_rel_hist_7d,1071707.0,1.030621e+00,0.658933,0.000869,0.698555,0.914155,1.205795,1.738754e+02
entry_rate_rel_hist_7d,1062784.0,1.106590e+00,73.527035,0.000039,0.856285,0.978616,1.106430,7.541961e+04


3. Cross-Visits Network Features

In [5]:
# =====================================
# 3. Cross-Visits Network Features
# =====================================

# -----------------------------
# 3.1 Normalize cross_visits
# -----------------------------
cv = cross_visits.copy()


cv["total_cross_visits"] = pd.to_numeric(cv["total_cross_visits"], errors="coerce")

cv_ab = cv.rename(columns={
    "store_code_1": "store_code",
    "store_code_2": "neighbor_store"
})

cv_ba = cv.rename(columns={
    "store_code_2": "store_code",
    "store_code_1": "neighbor_store"
})

cv_long = pd.concat([cv_ab, cv_ba], ignore_index=True)

# -----------------------------
# 3.2 Basic network strength
# -----------------------------
net_basic = (
    cv_long
    .groupby("store_code")
    .agg(
        cross_visit_degree=("neighbor_store", "nunique"),
        cross_visit_strength=("total_cross_visits", "sum"),
        cross_visit_mean=("total_cross_visits", "mean")
    )
    .reset_index()
)

# -----------------------------
# 3.3 Neighbor quality (store-level)
# -----------------------------
# use proxy（sales / sri）in store_master as long-term quality

neighbor_quality = (
    cv_long
    .merge(
        store_master[[
            "store_code",
            "sales_r12m",
            "sales_per_cost",
            "sri_score"
        ]],
        left_on="neighbor_store",
        right_on="store_code",
        how="left",
        suffixes=("", "_nbr")
    )
)

neighbor_agg = (
    neighbor_quality
    .groupby("store_code")
    .agg(
        nbr_sales_mean=("sales_r12m", "mean"),
        nbr_sales_per_cost_mean=("sales_per_cost", "mean"),
        nbr_sri_mean=("sri_score", "mean")
    )
    .reset_index()
)

# -----------------------------
# 3.4 Top-K neighbor intensity
# -----------------------------
K = 5

cv_long_sorted = cv_long.sort_values(
    ["store_code", "total_cross_visits"],
    ascending=[True, False]
)

topk = (
    cv_long_sorted
    .groupby("store_code")
    .head(K)
    .groupby("store_code")
    .agg(
        topk_cross_visits_sum=("total_cross_visits", "sum"),
        topk_cross_visits_mean=("total_cross_visits", "mean")
    )
    .reset_index()
)

# -----------------------------
# 3.5 Assemble network features
# -----------------------------
network_features = (
    net_basic
    .merge(neighbor_agg, on="store_code", how="left")
    .merge(topk, on="store_code", how="left")
)

print("Network features shape:", network_features.shape)
network_features.head()


# -----------------------------
# 3.6 Merge to store_day
# -----------------------------
store_day = store_day.merge(
    network_features,
    on="store_code",
    how="left"
)

# Availability flag
store_day["has_network"] = store_day["cross_visit_degree"].notna().astype(int)



Network features shape: (3283, 9)


4. Feature Selection & Export

In [6]:
# =====================================
# 4. Feature Selection & Export
# =====================================

# -----------------------------
# 4.1 define feature groups
# -----------------------------

ID_COLS = [
    "date",
    "store_code",
    "mall_id",
    "block_id"
]

BASE_FEATURES = [
    "people_in",
    "people_window_flow",
    "entry_rate",
    "store_average_dwell_time",
    "store_median_dwell_time",
    "shopping_average_dwell_time",
    "outdoor_flag"
]

TEMPORAL_FEATURES = [
    c for c in store_day.columns
    if (
        "roll" in c
        or "_chg_" in c
        or "_pctchg_" in c
        or "_cv_" in c
        or c.startswith("has_history_")
    )
]

RELATIVE_FEATURES = [
    c for c in store_day.columns
    if (
        c.startswith("rel_")
        or "_z_mall" in c
        or "_rel_peer" in c
        or "_rel_hist_" in c
    )
]

NETWORK_FEATURES = [
    "cross_visit_degree",
    "cross_visit_strength",
    "cross_visit_mean",
    "topk_cross_visits_sum",
    "topk_cross_visits_mean",
    "nbr_sales_mean",
    "nbr_sales_per_cost_mean",
    "nbr_sri_mean",
    "has_network"
]

STATIC_FEATURES = [
    "gla",
    "gla_category",
    "bl1_label",
    "bl2_label",
    "bl3_label",
    "has_financials",
    "has_sri"
]

# -----------------------------
# 4.2 feature sets
# -----------------------------

FEATURE_GROUPS = {
    "base": BASE_FEATURES,
    "temporal": TEMPORAL_FEATURES,
    "relative": RELATIVE_FEATURES,
    "network": NETWORK_FEATURES,
    "static": STATIC_FEATURES
}

FULL_FEATURE_SET = (
    BASE_FEATURES
    + TEMPORAL_FEATURES
    + RELATIVE_FEATURES
    + NETWORK_FEATURES
    + STATIC_FEATURES
)

print("Feature counts:")
for k, v in FEATURE_GROUPS.items():
    print(f"{k:10s}: {len(v)}")

print(f"\nTotal features (full): {len(FULL_FEATURE_SET)}")


Feature counts:
base      : 7
temporal  : 57
relative  : 10
network   : 9
static    : 7

Total features (full): 90


In [7]:

# 6. Pipeline Export (For Model.ipynb)
# ------------------------------------
# Save the feature-enriched master table for the modeling notebook.
output_file = 'feature_engineered_master.csv'
store_master.to_csv(output_file, index=False)
print(f"Master Table saved to: {output_file}")
print(f"Columns: {store_master.columns.tolist()}")



Master Table saved to: feature_engineered_master.csv
Columns: ['mall_id', 'block_id', 'block_type', 'store_code', 'store_name', 'retailer_code', 'bl1_label', 'bl2_label', 'bl3_label', 'gla', 'gla_category', 'cur_code', 'sales_r12m', 'total_costs_r12m', 'sales_per_cost', 'has_financials', 'sri_score', 'has_sri']
